# * VINSIGHT : Gross Adds
    Prepaid(GEO Channel) : DTAC รอพี่หมอ re-run ต้องไม่นับ Wholesales, AE

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000101CS' --Prepaid Gross Adds
        , 'TB1S000101CS' --Prepaid Gross Adds : TMH
        , 'DB1S000101CS' --Prepaid Gross Adds : DTAC
        , 'B1S000101CG' --Prepaid Gross Adds - GEO Channel
        , 'TB1S000101CG' --Prepaid Gross Adds : TMH - GEO Channel
        , 'DB1S000101CG' --Prepaid Gross Adds : DTAC - GEO Channel
        
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'TB2S010100CS' --Postpaid Gross Adds B2C : TMH
        , 'DB2S010100CS' --Postpaid Gross Adds B2C : DTAC
        , 'B2S020100CS' --Postpaid Gross Adds B2B
        , 'TB2S020100CS' --Postpaid Gross Adds B2B : TMH
        , 'DB2S020100CS' --Postpaid Gross Adds B2B : DTAC
        , 'B2S010100CG' --Postpaid Gross Adds B2C - GEO Channel
        , 'TB2S010100CG' --Postpaid Gross Adds B2C : TMH - GEO Channel
        , 'DB2S010100CG' --Postpaid Gross Adds B2C : DTAC - GEO Channel
        , 'B2S020100CG' --Postpaid Gross Adds B2B - GEO Channel
        , 'TB2S020100CG' --Postpaid Gross Adds B2B : TMH - GEO Channel
        , 'DB2S020100CG' --Postpaid Gross Adds B2B : DTAC - GEO Channel
        
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB3S000102CG' --TOL Gross Adds Connected : Consumer - GEO Channel
        , 'TB3S000102D1CS' --TOL Gross Adds Connected : Consumer (Install Location)
        , 'TB3S000102D1CG' --TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
        , 'TB3S000103CS' --TOL Gross Adds Connected : DataService
        , 'TB3S000103CG' --TOL Gross Adds Connected : DataService - GEO Channel
        , 'TB3S000103D1CS' --TOL Gross Adds Connected : DataService (Install Location)
        , 'TB3S000103D1CG' --TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
        , 'TB3S000104CS' --TOL Gross Adds Registration : Consumer
        , 'TB3S000104CG' --TOL Gross Adds Registration : Consumer - GEO Channel
        , 'TB3S000105CS' --TOL Gross Adds Registration : DataService
        , 'TB3S000105CG' --TOL Gross Adds Registration : DataService - GEO Channel
        
        , 'TB4S001004CS' --TVS CMDU Gross Adds
        , 'TB4S001004CG' --TVS CMDU Gross Adds - GEO Channel
        , 'TB4S001400CS' --TVS Now Gross Adds
        , 'TB4S001400CG' --TVS Now Gross Adds - GEO Channel
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-06-26, 10:29:57

DataFrame: 5984 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

C:\Users\Narut4\AppData\Local\Temp\ipykernel_11408\1881320084.py:4: RuntimeWarning: invalid value encountered in cast
  prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)


## Overview(All) by Period

In [4]:
''' Gross Adds(All Channel) Yearly '''

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_yearly_df = ga_yearly_df.loc[ga_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_yearly_df['PRE'] = np.where(ga_yearly_df['METRIC_CD']=='B1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['PRE_T'] = np.where(ga_yearly_df['METRIC_CD']=='TB1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['PRE_D'] = np.where(ga_yearly_df['METRIC_CD']=='DB1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_B2C'] = np.where(ga_yearly_df['METRIC_CD']=='B2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_B2C_T'] = np.where(ga_yearly_df['METRIC_CD']=='TB2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_B2C_D'] = np.where(ga_yearly_df['METRIC_CD']=='DB2S010100CS', ga_yearly_df['P'], 0)
# ga_yearly_df['POST_B2B'] = np.where(ga_yearly_df['METRIC_CD']=='B2S020100CS', ga_yearly_df['P'], 0)
# ga_yearly_df['POST_B2B_T'] = np.where(ga_yearly_df['METRIC_CD']=='TB2S020100CS', ga_yearly_df['P'], 0)
# ga_yearly_df['POST_B2B_D'] = np.where(ga_yearly_df['METRIC_CD']=='DB2S020100CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_REG_CON'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000104CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_CONN'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_CONN(Install)'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102D1CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_REG_DSV'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000105CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_DSV'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000103CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TOL_DSV(Install)'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000103D1CS', ga_yearly_df['P'], 0)
ga_yearly_df['TVS_NOW'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001400CS', ga_yearly_df['P'], 0)
# ga_yearly_df['TVS_CMDU'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001004CS', ga_yearly_df['P'], 0)

# ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_yearly_df_display = ga_yearly_df.copy()
ga_yearly_df_display['ACTUAL_AS_OF'] = ga_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_yearly_df_display[col] = ga_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025,2025-06-26 01:10:35,20250624,"11,789,946","5,599,885","6,190,434","1,088,269","828,426","259,843","397,547","289,366","8,977"


In [5]:
''' Gross Adds(All Channel) Quarterly '''

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['METRIC_CD'].isin(v_metric_list)]

ga_quarterly_df['PRE'] = np.where(ga_quarterly_df['METRIC_CD']=='B1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['PRE_T'] = np.where(ga_quarterly_df['METRIC_CD']=='TB1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['PRE_D'] = np.where(ga_quarterly_df['METRIC_CD']=='DB1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_B2C'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_B2C_T'] = np.where(ga_quarterly_df['METRIC_CD']=='TB2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_B2C_D'] = np.where(ga_quarterly_df['METRIC_CD']=='DB2S010100CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['POST_B2B'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S020100CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['POST_B2B_T'] = np.where(ga_quarterly_df['METRIC_CD']=='TB2S020100CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['POST_B2B_D'] = np.where(ga_quarterly_df['METRIC_CD']=='DB2S020100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_REG_CON'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000104CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_CONN'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_CONN(Install)'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102D1CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_REG_DSV'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000105CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_DSV'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000103CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TOL_DSV(Install)'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000103D1CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TVS_NOW'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001400CS', ga_quarterly_df['P'], 0)
# ga_quarterly_df['TVS_CMDU'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001004CS', ga_quarterly_df['P'], 0)

# ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_quarterly_df_display = ga_quarterly_df.copy()
ga_quarterly_df_display['ACTUAL_AS_OF'] = ga_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_quarterly_df_display[col] = ga_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,20251,2025-06-26 01:10:35,20250331,"6,673,218","3,141,270","3,531,948","569,340","438,816","130,524","198,847","145,692","3,337"
1,20252,2025-06-26 01:10:35,20250624,"5,116,728","2,458,615","2,658,486","518,929","389,610","129,319","198,700","143,674","5,640"


In [6]:
''' Gross Adds(All Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['METRIC_CD'].isin(v_metric_list)]

ga_monthly_df['PRE'] = np.where(ga_monthly_df['METRIC_CD']=='B1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['PRE_T'] = np.where(ga_monthly_df['METRIC_CD']=='TB1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['PRE_D'] = np.where(ga_monthly_df['METRIC_CD']=='DB1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_B2C'] = np.where(ga_monthly_df['METRIC_CD']=='B2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_B2C_T'] = np.where(ga_monthly_df['METRIC_CD']=='TB2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_B2C_D'] = np.where(ga_monthly_df['METRIC_CD']=='DB2S010100CS', ga_monthly_df['P'], 0)
# ga_monthly_df['POST_B2B'] = np.where(ga_monthly_df['METRIC_CD']=='B2S020100CS', ga_monthly_df['P'], 0)
# ga_monthly_df['POST_B2B_T'] = np.where(ga_monthly_df['METRIC_CD']=='TB2S020100CS', ga_monthly_df['P'], 0)
# ga_monthly_df['POST_B2B_D'] = np.where(ga_monthly_df['METRIC_CD']=='DB2S020100CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_REG_CON'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000104CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_CONN'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_CONN(Install)'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102D1CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_REG_DSV'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000105CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_DSV'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000103CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TOL_DSV(Install)'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000103D1CS', ga_monthly_df['P'], 0)
ga_monthly_df['TVS_NOW'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001400CS', ga_monthly_df['P'], 0)
# ga_monthly_df['TVS_CMDU'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001004CS', ga_monthly_df['P'], 0)

# ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_monthly_df_display = ga_monthly_df.copy()
ga_monthly_df_display['ACTUAL_AS_OF'] = ga_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_monthly_df_display[col] = ga_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202501,2025-06-26 01:10:35,20250131,"2,507,456","1,166,199","1,341,257","188,248","142,922","45,326","64,026","46,939",769
1,202502,2025-06-26 01:10:35,20250228,"2,161,914","999,208","1,162,706","183,973","140,856","43,117","62,508","45,846",545
2,202503,2025-06-26 01:10:35,20250331,"2,003,848","975,863","1,027,985","197,119","155,038","42,081","72,313","52,907","2,023"
3,202504,2025-06-26 01:10:35,20250430,"1,879,322","931,089","948,233","197,270","152,200","45,070","66,846","48,482","2,352"
4,202505,2025-06-26 01:10:35,20250531,"1,912,819","912,219","1,000,600","182,723","134,956","47,767","72,984","52,741","2,206"
5,202506,2025-06-26 01:10:35,20250624,"1,324,587","615,307","709,653","138,936","102,454","36,482","58,870","42,451","1,082"


In [7]:
''' Gross Adds(All Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['METRIC_CD'].isin(v_metric_list)]

ga_weekly_df['PRE'] = np.where(ga_weekly_df['METRIC_CD']=='B1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['PRE_T'] = np.where(ga_weekly_df['METRIC_CD']=='TB1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['PRE_D'] = np.where(ga_weekly_df['METRIC_CD']=='DB1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_B2C'] = np.where(ga_weekly_df['METRIC_CD']=='B2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_B2C_T'] = np.where(ga_weekly_df['METRIC_CD']=='TB2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_B2C_D'] = np.where(ga_weekly_df['METRIC_CD']=='DB2S010100CS', ga_weekly_df['P'], 0)
# ga_weekly_df['POST_B2B'] = np.where(ga_weekly_df['METRIC_CD']=='B2S020100CS', ga_weekly_df['P'], 0)
# ga_weekly_df['POST_B2B_T'] = np.where(ga_weekly_df['METRIC_CD']=='TB2S020100CS', ga_weekly_df['P'], 0)
# ga_weekly_df['POST_B2B_D'] = np.where(ga_weekly_df['METRIC_CD']=='DB2S020100CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_REG_CON'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000104CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_CONN'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_CONN(Install)'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102D1CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_REG_DSV'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000105CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_DSV'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000103CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TOL_DSV(Install)'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000103D1CS', ga_weekly_df['P'], 0)
ga_weekly_df['TVS_NOW'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001400CS', ga_weekly_df['P'], 0)
# ga_weekly_df['TVS_CMDU'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001004CS', ga_weekly_df['P'], 0)

# ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_weekly_df_display = ga_weekly_df.copy()
ga_weekly_df_display['ACTUAL_AS_OF'] = ga_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_weekly_df_display[col] = ga_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025018,2025-06-26 01:10:35,20250504,"275,839","132,391","143,448","26,324","20,205","6,119","9,079","5,718",294
1,2025019,2025-06-26 01:10:35,20250511,"440,498","216,038","224,460","45,660","34,203","11,457","17,702","12,881",397
2,2025020,2025-06-26 01:10:35,20250518,"424,641","217,785","206,856","43,160","32,354","10,806","16,822","12,288",354
3,2025021,2025-06-26 01:10:35,20250525,"407,397","183,328","224,069","36,559","25,537","11,022","15,854","11,961",484
4,2025022,2025-06-26 01:10:35,20250601,"429,074","190,922","238,152","36,591","27,260","9,331","15,560","11,563",746
5,2025023,2025-06-26 01:10:35,20250608,"421,599","192,353","229,246","42,351","32,135","10,216","17,230","12,379",333
6,2025024,2025-06-26 01:10:35,20250615,"387,129","183,698","203,431","40,541","30,697","9,844","16,958","11,599",277
7,2025025,2025-06-26 01:10:35,20250622,"396,623","185,078","211,545","39,737","27,223","12,514","17,293","13,108",346
8,2025026,2025-06-26 01:10:35,20250624,"54,606","25,933","29,046","10,736","7,796","2,940","5,356","3,695",57


In [8]:
''' Gross Adds(All Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CS' #Prepaid Gross Adds
	, 'TB1S000101CS' #Prepaid Gross Adds : TMH
	, 'DB1S000101CS' #Prepaid Gross Adds : DTAC
	
	, 'B2S010100CS' #Postpaid Gross Adds B2C
	, 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
	, 'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
	# , 'B2S020100CS' #Postpaid Gross Adds B2B
	# , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
	# , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
	
	, 'TB3S000104CS' #TOL Gross Adds Registration : Consumer	
	, 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
	# , 'TB3S000102D1CS' #TOL Gross Adds Connected : Consumer (Install Location)
    # , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
	# , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
	# , 'TB3S000103D1CS' #TOL Gross Adds Connected : DataService (Install Location)
	
    , 'TB4S001400CS' #TVS Now Gross Adds
	# , 'TB4S001004CS' #TVS CMDU Gross Adds
    ]

ga_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_daily_df = ga_daily_df.loc[ga_daily_df['METRIC_CD'].isin(v_metric_list)]

ga_daily_df['PRE'] = np.where(ga_daily_df['METRIC_CD']=='B1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_T'] = np.where(ga_daily_df['METRIC_CD']=='TB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['PRE_D'] = np.where(ga_daily_df['METRIC_CD']=='DB1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C'] = np.where(ga_daily_df['METRIC_CD']=='B2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['POST_B2C_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S010100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CS', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_REG_CON'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000104CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_CONN'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_CONN(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102D1CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_REG_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000105CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103CS', ga_daily_df['P'], 0)
# ga_daily_df['TOL_DSV(Install)'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000103D1CS', ga_daily_df['P'], 0)
ga_daily_df['TVS_NOW'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001400CS', ga_daily_df['P'], 0)
# ga_daily_df['TVS_CMDU'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001004CS', ga_daily_df['P'], 0)

# ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
# ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_daily_df_display = ga_daily_df.copy()
mod_col_list = ga_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_daily_df_display[col] = ga_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202506,20250625,2025-06-26 01:10:35,0,0,0,0,0,0,0,0,0
1,202506,20250624,2025-06-26 01:10:35,0,373,0,"5,737","4,280","1,457","2,763","1,809",0
2,202506,20250623,2025-06-26 01:10:35,"54,606","25,560","29,046","4,999","3,516","1,483","2,593","1,886",57
3,202506,20250622,2025-06-26 01:10:35,"54,626","24,901","29,725","5,646","4,015","1,631","1,968","1,730",65
4,202506,20250621,2025-06-26 01:10:35,"57,522","26,491","31,031","5,396","3,978","1,418","2,407","1,977",70
5,202506,20250620,2025-06-26 01:10:35,"64,910","30,091","34,819","5,290","3,748","1,542","2,633","1,819",46
6,202506,20250619,2025-06-26 01:10:35,"59,323","27,855","31,468","5,402","3,722","1,680","2,628","1,774",40
7,202506,20250618,2025-06-26 01:10:35,"53,984","26,190","27,794","5,391","3,591","1,800","2,544","1,867",49
8,202506,20250617,2025-06-26 01:10:35,"52,215","24,211","28,004","6,995","4,428","2,567","2,688","1,917",44
9,202506,20250616,2025-06-26 01:10:35,"54,043","25,339","28,704","5,617","3,741","1,876","2,425","2,024",32


## Overview(Geo) by Period

In [9]:
''' Gross Adds(Geo Channel) Yearly '''

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_geo_yearly_df = ga_geo_yearly_df.loc[ga_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_yearly_df['PRE'] = np.where(ga_geo_yearly_df['METRIC_CD']=='B1S000101CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['PRE_T'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB1S000101CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['PRE_D'] = np.where(ga_geo_yearly_df['METRIC_CD']=='DB1S000101CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['POST_B2C'] = np.where(ga_geo_yearly_df['METRIC_CD']=='B2S010100CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['POST_B2C_T'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB2S010100CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['POST_B2C_D'] = np.where(ga_geo_yearly_df['METRIC_CD']=='DB2S010100CG', ga_geo_yearly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_yearly_df['TOL_REG_CON'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000104CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['TOL_CONN'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000102CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_CONN(Install)'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_REG_DSV'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000105CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_DSV'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000103CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TOL_DSV(Install)'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_yearly_df['P'], 0)
ga_geo_yearly_df['TVS_NOW'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB4S001400CG', ga_geo_yearly_df['P'], 0)
# ga_geo_yearly_df['TVS_CMDU'] = np.where(ga_geo_yearly_df['METRIC_CD']=='TB4S001004CG', ga_geo_yearly_df['P'], 0)

# ga_geo_yearly_df = ga_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_yearly_df = ga_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
# ga_geo_yearly_df = ga_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_yearly_df = ga_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_yearly_df = ga_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_geo_yearly_df = ga_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_yearly_df_display = ga_geo_yearly_df.copy()
ga_geo_yearly_df_display['ACTUAL_AS_OF'] = ga_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_yearly_df_display[col] = ga_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025,2025-06-26 01:10:35,20250624,"9,992,281","5,580,301","4,412,353","490,299","413,213","77,086","303,318","224,105","8,954"


In [10]:
''' Gross Adds(Geo Channel) Quarterly '''

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
ga_geo_quarterly_df = ga_geo_quarterly_df.loc[ga_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_quarterly_df['PRE'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='B1S000101CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['PRE_T'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB1S000101CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['PRE_D'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='DB1S000101CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['POST_B2C'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='B2S010100CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['POST_B2C_T'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB2S010100CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['POST_B2C_D'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='DB2S010100CG', ga_geo_quarterly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_quarterly_df['TOL_REG_CON'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000104CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['TOL_CONN'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000102CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_CONN(Install)'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_REG_DSV'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000105CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_DSV'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000103CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TOL_DSV(Install)'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_quarterly_df['P'], 0)
ga_geo_quarterly_df['TVS_NOW'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB4S001400CG', ga_geo_quarterly_df['P'], 0)
# ga_geo_quarterly_df['TVS_CMDU'] = np.where(ga_geo_quarterly_df['METRIC_CD']=='TB4S001004CG', ga_geo_quarterly_df['P'], 0)

# ga_geo_quarterly_df = ga_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_quarterly_df = ga_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
# ga_geo_quarterly_df = ga_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_quarterly_df = ga_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_quarterly_df = ga_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_geo_quarterly_df = ga_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_quarterly_df_display = ga_geo_quarterly_df.copy()
ga_geo_quarterly_df_display['ACTUAL_AS_OF'] = ga_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_quarterly_df_display[col] = ga_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,20251,2025-06-26 01:10:35,20250331,"5,553,809","3,132,044","2,421,765","262,488","223,283","39,205","150,337","111,869","3,329"
1,20252,2025-06-26 01:10:35,20250624,"4,438,472","2,448,257","1,990,588","227,811","189,930","37,881","152,981","112,236","5,625"


In [11]:
''' Gross Adds(Geo Channel) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
ga_geo_monthly_df = ga_geo_monthly_df.loc[ga_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_monthly_df['PRE'] = np.where(ga_geo_monthly_df['METRIC_CD']=='B1S000101CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['PRE_T'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB1S000101CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['PRE_D'] = np.where(ga_geo_monthly_df['METRIC_CD']=='DB1S000101CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['POST_B2C'] = np.where(ga_geo_monthly_df['METRIC_CD']=='B2S010100CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['POST_B2C_T'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB2S010100CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['POST_B2C_D'] = np.where(ga_geo_monthly_df['METRIC_CD']=='DB2S010100CG', ga_geo_monthly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_monthly_df['TOL_REG_CON'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000104CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['TOL_CONN'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000102CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_CONN(Install)'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_REG_DSV'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000105CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_DSV'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000103CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TOL_DSV(Install)'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_monthly_df['P'], 0)
ga_geo_monthly_df['TVS_NOW'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB4S001400CG', ga_geo_monthly_df['P'], 0)
# ga_geo_monthly_df['TVS_CMDU'] = np.where(ga_geo_monthly_df['METRIC_CD']=='TB4S001004CG', ga_geo_monthly_df['P'], 0)

# ga_geo_monthly_df = ga_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_monthly_df = ga_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
# ga_geo_monthly_df = ga_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_monthly_df = ga_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_monthly_df = ga_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_geo_monthly_df = ga_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_monthly_df_display = ga_geo_monthly_df.copy()
ga_geo_monthly_df_display['ACTUAL_AS_OF'] = ga_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_monthly_df_display[col] = ga_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202501,2025-06-26 01:10:35,20250131,"2,038,766","1,163,218","875,548","83,267","70,852","12,415","46,010","34,626",765
1,202502,2025-06-26 01:10:35,20250228,"1,798,194","996,201","801,993","83,956","71,425","12,531","48,150","35,785",543
2,202503,2025-06-26 01:10:35,20250331,"1,716,849","972,625","744,224","95,265","81,006","14,259","56,177","41,458","2,021"
3,202504,2025-06-26 01:10:35,20250430,"1,636,131","926,446","709,685","92,697","78,554","14,143","51,632","37,924","2,349"
4,202505,2025-06-26 01:10:35,20250531,"1,662,837","908,569","754,268","78,554","64,712","13,842","56,167","41,428","2,196"
5,202506,2025-06-26 01:10:35,20250624,"1,139,504","613,242","526,635","56,560","46,664","9,896","45,182","32,884","1,080"


In [12]:
''' Gross Adds(Geo Channel) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_geo_weekly_df = ga_geo_weekly_df.loc[ga_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_weekly_df['PRE'] = np.where(ga_geo_weekly_df['METRIC_CD']=='B1S000101CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['PRE_T'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB1S000101CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['PRE_D'] = np.where(ga_geo_weekly_df['METRIC_CD']=='DB1S000101CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['POST_B2C'] = np.where(ga_geo_weekly_df['METRIC_CD']=='B2S010100CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['POST_B2C_T'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB2S010100CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['POST_B2C_D'] = np.where(ga_geo_weekly_df['METRIC_CD']=='DB2S010100CG', ga_geo_weekly_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_weekly_df['TOL_REG_CON'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000104CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['TOL_CONN'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000102CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_CONN(Install)'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_REG_DSV'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000105CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_DSV'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000103CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TOL_DSV(Install)'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_weekly_df['P'], 0)
ga_geo_weekly_df['TVS_NOW'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB4S001400CG', ga_geo_weekly_df['P'], 0)
# ga_geo_weekly_df['TVS_CMDU'] = np.where(ga_geo_weekly_df['METRIC_CD']=='TB4S001004CG', ga_geo_weekly_df['P'], 0)

# ga_geo_weekly_df = ga_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_weekly_df = ga_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
# ga_geo_weekly_df = ga_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_weekly_df = ga_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_weekly_df = ga_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_geo_weekly_df = ga_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_weekly_df_display = ga_geo_weekly_df.copy()
ga_geo_weekly_df_display['ACTUAL_AS_OF'] = ga_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = ga_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_weekly_df_display[col] = ga_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,2025018,2025-06-26 01:10:35,20250504,"235,930","131,910","104,020","12,361","10,111","2,250","7,261","4,542",293
1,2025019,2025-06-26 01:10:35,20250511,"387,557","215,024","172,533","19,935","16,622","3,313","13,666","10,275",396
2,2025020,2025-06-26 01:10:35,20250518,"374,653","216,958","157,695","18,580","15,407","3,173","12,598","9,600",354
3,2025021,2025-06-26 01:10:35,20250525,"354,381","182,635","171,746","15,339","12,498","2,841","12,359","9,352",480
4,2025022,2025-06-26 01:10:35,20250601,"367,475","190,188","177,287","15,099","12,226","2,873","11,892","8,899",742
5,2025023,2025-06-26 01:10:35,20250608,"367,486","191,795","175,691","17,547","14,339","3,208","13,355","9,632",332
6,2025024,2025-06-26 01:10:35,20250615,"332,217","182,925","149,292","16,293","13,613","2,680","12,969","9,107",276
7,2025025,2025-06-26 01:10:35,20250622,"336,591","184,487","152,104","15,545","12,903","2,642","13,309","10,085",346
8,2025026,2025-06-26 01:10:35,20250624,"46,051","25,889","20,535","4,415","3,657",758,"3,940","2,820",57


In [13]:
''' Gross Adds(Geo Channel) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
	'B1S000101CG' #Prepaid Gross Adds - GEO Channel
	, 'TB1S000101CG' #Prepaid Gross Adds : TMH - GEO Channel
	, 'DB1S000101CG' #Prepaid Gross Adds : DTAC - GEO Channel
	
	, 'B2S010100CG' #Postpaid Gross Adds B2C - GEO Channel
	, 'TB2S010100CG' #Postpaid Gross Adds B2C : TMH - GEO Channel
	, 'DB2S010100CG' #Postpaid Gross Adds B2C : DTAC - GEO Channel
	# , 'B2S020100CG' #Postpaid Gross Adds B2B - GEO Channel
	# , 'TB2S020100CG' #Postpaid Gross Adds B2B : TMH - GEO Channel
	# , 'DB2S020100CG' #Postpaid Gross Adds B2B : DTAC - GEO Channel

	, 'TB3S000104CG' #TOL Gross Adds Registration : Consumer - GEO Channel	
	, 'TB3S000102CG' #TOL Gross Adds Connected : Consumer - GEO Channel
	# , 'TB3S000102D1CG' #TOL Gross Adds Connected : Consumer - GEO Channel (Install Location)
    # , 'TB3S000105CG' #TOL Gross Adds Registration : DataService - GEO Channel
	# , 'TB3S000103CG' #TOL Gross Adds Connected : DataService - GEO Channel
	# , 'TB3S000103D1CG' #TOL Gross Adds Connected : DataService - GEO Channel (Install Location)
	
	, 'TB4S001400CG' #TVS Now Gross Adds - GEO Channel
	# , 'TB4S001004CG' #TVS CMDU Gross Adds - GEO Channel
    ]

ga_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_geo_daily_df = ga_geo_daily_df.loc[ga_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

ga_geo_daily_df['PRE'] = np.where(ga_geo_daily_df['METRIC_CD']=='B1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['PRE_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB1S000101CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C'] = np.where(ga_geo_daily_df['METRIC_CD']=='B2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_T'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB2S010100CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['POST_B2C_D'] = np.where(ga_geo_daily_df['METRIC_CD']=='DB2S010100CG', ga_geo_daily_df['P'], 0)
# ga_daily_df['POST_B2B'] = np.where(ga_daily_df['METRIC_CD']=='B2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_T'] = np.where(ga_daily_df['METRIC_CD']=='TB2S020100CG', ga_daily_df['P'], 0)
# ga_daily_df['POST_B2B_D'] = np.where(ga_daily_df['METRIC_CD']=='DB2S020100CG', ga_daily_df['P'], 0)
ga_geo_daily_df['TOL_REG_CON'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000104CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TOL_CONN'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_CONN(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000102D1CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_REG_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000105CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TOL_DSV(Install)'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB3S000103D1CG', ga_geo_daily_df['P'], 0)
ga_geo_daily_df['TVS_NOW'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001400CG', ga_geo_daily_df['P'], 0)
# ga_geo_daily_df['TVS_CMDU'] = np.where(ga_geo_daily_df['METRIC_CD']=='TB4S001004CG', ga_geo_daily_df['P'], 0)

# ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'POST_B2B':'sum', 'POST_B2B_T':'sum', 'POST_B2B_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TOL_CONN(Install)':'sum', 'TOL_REG_DSV':'sum', 'TOL_DSV':'sum', 'TOL_DSV(Install)':'sum', 'TVS_NOW':'sum', 'TVS_CMDU':'sum'})
# ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
# ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'POST_B2B', 'POST_B2B_T', 'POST_B2B_D', 'TOL_REG_CON', 'TOL_CONN', 'TOL_CONN(Install)', 'TOL_REG_DSV', 'TOL_DSV', 'TOL_DSV(Install)', 'TVS_NOW', 'TVS_CMDU']]

ga_geo_daily_df = ga_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL_REG_CON':'sum', 'TOL_CONN':'sum', 'TVS_NOW':'sum'})
ga_geo_daily_df = ga_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_geo_daily_df = ga_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL_REG_CON', 'TOL_CONN', 'TVS_NOW']]

ga_geo_daily_df_display = ga_geo_daily_df.copy()
mod_col_list = ga_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_geo_daily_df_display[col] = ga_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
ga_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL_REG_CON,TOL_CONN,TVS_NOW
0,202506,20250625,2025-06-26 01:10:35,0,0,0,0,0,0,0,0,0
1,202506,20250624,2025-06-26 01:10:35,0,373,0,"2,313","1,923",390,"2,018","1,357",0
2,202506,20250623,2025-06-26 01:10:35,"46,051","25,516","20,535","2,102","1,734",368,"1,922","1,463",57
3,202506,20250622,2025-06-26 01:10:35,"46,654","24,829","21,825","2,241","1,821",420,"1,524","1,241",65
4,202506,20250621,2025-06-26 01:10:35,"48,575","26,420","22,155","2,310","1,910",400,"1,931","1,437",70
5,202506,20250620,2025-06-26 01:10:35,"54,905","30,012","24,893","2,136","1,813",323,"2,036","1,417",46
6,202506,20250619,2025-06-26 01:10:35,"49,855","27,751","22,104","2,167","1,814",353,"1,958","1,386",40
7,202506,20250618,2025-06-26 01:10:35,"45,682","26,095","19,587","2,078","1,743",335,"1,939","1,476",49
8,202506,20250617,2025-06-26 01:10:35,"44,999","24,111","20,888","2,516","2,061",455,"2,022","1,541",44
9,202506,20250616,2025-06-26 01:10:35,"45,921","25,269","20,652","2,097","1,741",356,"1,899","1,587",32


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202506


### Prepaid

In [16]:
''' Prepaid (All Channel) '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[~prepaid_df['METRIC_NAME'].str.contains('GEO')]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Prepaid,B1S000101CS,Prepaid Gross Adds,2025-06-26 01:10:35,"1,324,587","1,324,587","1,162,335","1,162,335","1,135,971"
1,202506,Prepaid,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-06-26 01:10:35,"709,653","709,653","692,712","692,712","677,939"
2,202506,Prepaid,TB1S000101CS,Prepaid Gross Adds : TMH,2025-06-26 01:10:35,"615,307","615,307","469,737","469,737","458,144"


In [17]:
''' Prepaid (Geo Channel) '''

v_product_grp = 'Prepaid'

prepaid_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['METRIC_NAME'].str.contains('GEO')]
prepaid_geo_df = prepaid_geo_df.loc[prepaid_geo_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_geo_df = prepaid_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
prepaid_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Prepaid,B1S000101CG,Prepaid Gross Adds - GEO Channel,2025-06-26 01:10:35,0,"1,139,504","990,656","990,656","965,587"
1,202506,Prepaid,DB1S000101CG,Prepaid Gross Adds : DTAC - GEO Channel,2025-06-26 01:10:35,0,"526,635","521,201","521,201","507,723"
2,202506,Prepaid,TB1S000101CG,Prepaid Gross Adds : TMH - GEO Channel,2025-06-26 01:10:35,0,"613,242","469,569","469,569","457,976"


### Postpaid

In [18]:
''' Postpaid B2C (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[~postpaid_b2c_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Postpaid,B2S010100CS,Postpaid Gross Adds B2C,2025-06-26 01:10:35,"138,936","138,936","104,438","104,438","104,413"
1,202506,Postpaid,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-06-26 01:10:35,"36,482","36,482","36,269","36,269","36,269"
2,202506,Postpaid,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-06-26 01:10:35,"102,454","102,454","68,169","68,169","68,169"


In [19]:
''' Postpaid B2C (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2c_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.loc[postpaid_b2c_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_geo_df = postpaid_b2c_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2c_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Postpaid,B2S010100CG,Postpaid Gross Adds B2C - GEO Channel,2025-06-26 01:10:35,0,"56,560","56,540","56,374","54,682"
1,202506,Postpaid,DB2S010100CG,Postpaid Gross Adds B2C : DTAC - GEO Channel,2025-06-26 01:10:35,0,"9,896","9,877","9,877","9,877"
2,202506,Postpaid,TB2S010100CG,Postpaid Gross Adds B2C : TMH - GEO Channel,2025-06-26 01:10:35,0,"46,664","46,663","46,663","46,665"


In [20]:
''' Postpaid B2B (All Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[~postpaid_b2b_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Postpaid,B2S020100CS,Postpaid Gross Adds B2B,2025-06-26 01:10:35,"27,323","27,323","5,459","5,095","4,972"
1,202506,Postpaid,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-06-26 01:10:35,"7,119","7,119","2,832","2,832","2,832"
2,202506,Postpaid,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-06-26 01:10:35,"20,204","20,204","2,642","2,644","2,644"


In [21]:
''' Postpaid B2B (Geo Channel) '''

v_product_grp = 'Postpaid'

postpaid_b2b_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['METRIC_NAME'].str.contains('GEO')]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.loc[postpaid_b2b_geo_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_geo_df = postpaid_b2b_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
postpaid_b2b_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,Postpaid,B2S020100CG,Postpaid Gross Adds B2B - GEO Channel,2025-06-26 01:10:35,0,"3,370","3,349","2,652","2,766"
1,202506,Postpaid,DB2S020100CG,Postpaid Gross Adds B2B : DTAC - GEO Channel,2025-06-26 01:10:35,0,734,734,734,734
2,202506,Postpaid,TB2S020100CG,Postpaid Gross Adds B2B : TMH - GEO Channel,2025-06-26 01:10:35,0,"2,636","2,636","2,638","2,638"


### TOL

In [22]:
''' TOL Registration : Consumer (All & Geo) '''

v_product_grp = 'TOL'

tol_reg_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['METRIC_NAME'].str.contains('Registration')]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['METRIC_NAME'].str.contains('Consumer')]
# tol_reg_con_df = tol_reg_con_df.loc[~tol_reg_con_df['METRIC_NAME'].str.contains('GEO')]
tol_reg_con_df = tol_reg_con_df.loc[tol_reg_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_reg_con_df = tol_reg_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_reg_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-06-26 01:10:35,"58,870","58,870","45,174","45,174","44,682"
1,202506,TOL,TB3S000104CG,TOL Gross Adds Registration : Consumer - GEO C...,2025-06-26 01:10:35,0,"45,182","45,169","45,169","44,677"


In [23]:
''' TOL Registration : DataService (All & Geo) '''

v_product_grp = 'TOL'

tol_reg_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['METRIC_NAME'].str.contains('Registration')]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['METRIC_NAME'].str.contains('DataService')]
# tol_reg_dsv_df = tol_reg_dsv_df.loc[~tol_reg_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_reg_dsv_df = tol_reg_dsv_df.loc[tol_reg_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_reg_dsv_df = tol_reg_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_reg_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-06-26 01:10:35,"1,325","1,325",43,43,43
1,202506,TOL,TB3S000105CG,TOL Gross Adds Registration : DataService - GE...,2025-06-26 01:10:35,0,43,43,43,43


In [24]:
''' TOL Connected : Consumer (All Channel) '''

v_product_grp = 'TOL'

tol_conn_con_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_df = tol_conn_con_df.loc[~tol_conn_con_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_df = tol_conn_con_df.loc[tol_conn_con_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_df = tol_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"42,451","42,451","32,883","32,883","32,517"
1,202506,TOL,TB3S000102D1CS,TOL Gross Adds Connected : Consumer (Install L...,2025-06-26 01:10:35,"42,451","42,451","42,313","42,313","42,339"


In [25]:
''' TOL Connected : Consumer (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_con_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('Consumer')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_con_geo_df = tol_conn_con_geo_df.loc[tol_conn_con_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_con_geo_df = tol_conn_con_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_con_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3S000102CG,TOL Gross Adds Connected : Consumer - GEO Channel,2025-06-26 01:10:35,0,"32,884","32,882","32,882","32,516"
1,202506,TOL,TB3S000102D1CG,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-06-26 01:10:35,0,"32,884","32,883","32,883","32,909"


In [26]:
''' TOL Connected : DataService (All Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[~tol_conn_dsv_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_df = tol_conn_dsv_df.loc[tol_conn_dsv_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_df = tol_conn_dsv_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-06-26 01:10:35,"1,320","1,320",29,29,29
1,202506,TOL,TB3S000103D1CS,TOL Gross Adds Connected : DataService (Instal...,2025-06-26 01:10:35,"1,320","1,320",29,29,29


In [27]:
''' TOL Connected : DataService (Geo Channel) '''

v_product_grp = 'TOL'

tol_conn_dsv_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('Connected')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('DataService')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[~tol_conn_dsv_geo_df['METRIC_NAME'].str.contains('GEO')]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.loc[tol_conn_dsv_geo_df['TM_KEY_MTH']==v_tm_key_mth]
tol_conn_dsv_geo_df = tol_conn_dsv_geo_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tol_conn_dsv_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-06-26 01:10:35,"1,320","1,320",29,29,29
1,202506,TOL,TB3S000103D1CS,TOL Gross Adds Connected : DataService (Instal...,2025-06-26 01:10:35,"1,320","1,320",29,29,29


### TVS

In [28]:
''' TVS Now '''

v_product_grp = 'TVS'

tvs_now_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_now_df = tvs_now_df.loc[tvs_now_df['METRIC_NAME'].str.contains('Now')]
tvs_now_df = tvs_now_df.loc[tvs_now_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_now_df = tvs_now_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_now_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TVS,TB4S001400CS,TVS Now Gross Adds,2025-06-26 01:10:35,"1,082","1,082","1,047","1,331","1,356"
1,202506,TVS,TB4S001400CG,TVS Now Gross Adds - GEO Channel,2025-06-26 01:10:35,0,"1,080","1,047","1,331","1,356"


In [29]:
''' TVS CMDU '''

v_product_grp = 'TVS'

tvs_cmdu_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['METRIC_NAME'].str.contains('CMDU')]
tvs_cmdu_df = tvs_cmdu_df.loc[tvs_cmdu_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_cmdu_df = tvs_cmdu_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
tvs_cmdu_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202506,TVS,TB4S001004CS,TVS CMDU Gross Adds,2025-06-26 01:10:35,32,32,34,34,34
1,202506,TVS,TB4S001004CG,TVS CMDU Gross Adds - GEO Channel,2025-06-26 01:10:35,0,1,1,1,1


## ** Issue History

In [30]:
''' Monthly : TOL Gross Adds Connected : Consumer '''

v_metric_cd = 'TB3S000102CS'

issue_tol_ga_conn_con_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_tol_ga_conn_con_df = issue_tol_ga_conn_con_df.loc[issue_tol_ga_conn_con_df['TM_KEY_MTH']>=202401]
issue_tol_ga_conn_con_df = issue_tol_ga_conn_con_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_tol_ga_conn_con_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH
0,202501,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"46,939","46,939","34,630","34,630","36,115"
1,202502,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"45,846","45,846","35,794","35,319","35,506"
2,202503,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"52,907","52,907","41,460","41,460","41,080"
3,202504,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"48,482","48,482","37,922","37,883","37,550"
4,202505,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"52,741","52,741","41,427","41,427","41,039"
5,202506,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"42,451","42,451","32,883","32,883","32,517"


In [31]:
''' Daily : TOL Gross Adds Connected : Consumer '''

v_metict_cd = 'TB3S000102CS'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_tol_conn_con_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_tol_conn_con_df = issue_daily_tol_conn_con_df.loc[issue_daily_tol_conn_con_df['METRIC_CD']==v_metict_cd]
issue_daily_tol_conn_con_df = issue_daily_tol_conn_con_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_tol_conn_con_df_display = issue_daily_tol_conn_con_df.copy()
mod_col_list = issue_daily_tol_conn_con_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_tol_conn_con_df_display[col] = issue_daily_tol_conn_con_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_tol_conn_con_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202506,20250625,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,nan,nan,nan,nan
1,202506,20250624,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"1,809","1,357","1,357","1,340"
2,202506,20250623,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"1,886","1,463","1,463","1,433"
3,202506,20250622,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"1,730","1,240","1,240","1,230"
4,202506,20250621,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"1,977","1,437","1,437","1,414"
5,202506,20250620,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"1,819","1,417","1,417","1,395"
6,202506,20250619,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"1,774","1,386","1,386","1,374"
7,202506,20250618,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"1,867","1,476","1,476","1,466"
8,202506,20250617,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"1,917","1,542","1,542","1,527"
9,202506,20250616,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-06-26 01:10:35,"2,024","1,587","1,587","1,574"


## ** Current Issue